In [65]:
import math
import random
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

In [66]:
df = pd.read_csv("../wavData.csv")
#drop columns unimportant to modeling
df.drop(['filename', 'frameID'], axis=1, inplace=True)
df['intelligence'] = df['intelligence'].map({"Human": 0, "AI": 1})
df.drop(['intelligence'], axis=1, inplace=False)
csv_data = df.values.tolist()

In [67]:
#split up training and test data 
#8planned 80/20 split
def splitData(data, ratio= 0.8):
    train_num = int(len(data) * ratio)
    training = []
    testing = list(data)
    while len(training) < train_num:
        index = random.randrange(len(testing))
        training.append(testing.pop(index))
    return training, testing

#Groups the data by class, key = class value = list of data for that class 
def groupByClass(data):
    ddict = {}
    for i in range(len(data)):
        if data[i][-1] not in ddict:
            ddict[data[i][-1]] = []
        ddict[data[i][-1]]
    return ddict

In [68]:
#calculate mean 
def Mean(num):
    m = np.mean(num)
    return m

#calculate standard deviation
def stdDev(num):
    stdev = np.std(num)
    return stdev

#calculate mean and standard deviation for a given class 
def classMeanAndStdDev(data):
    info = {}
    ddict = groupByClass(data)
    for cv, istnce in ddict.items():
        info[cv] = [classMeanAndStdDev(attribute) for attribute in zip(*istnce)]
    return info

In [69]:
#Calulate the Probability
def Probability(x, mean, stdev):
    expo = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(2 * stdev, 2))))
    retval = (1 / (math.sqrt(2 * math.pi) * (stdev))) * expo
    return retval

#claculates the Gaussian probabilty given a class 
def ClassGProbability(infoDict, testData):
    pb = {} # dictonary to store the probabilites 
    for cv, cs in infoDict.items():
        pb[cv] = 1
        for i in range(len(cs)):
            mean, std_dev = cs[i]
            x = testData[i]
            pb[cv] *= Probability(x, mean, std_dev)
    return pb

In [70]:
#get the class with the higest probability 
def ClassWithHighestProb(infoDict, test):
    probability = ClassGProbability(infoDict,test)
    hprob = max(probability, probability.get)
    return hprob

In [71]:
#Classify the data then make values into deciamls
def ClassifyData():
    dta = groupByClass(csv_data)
    for i in range(len(dta)):
        for j in range(len(dta[i]) - 1):
            dta[i][j] = float(dta[i][j])

In [72]:
training, test = splitData(csv_data,ratio=0.8)
print("Amount of training data",len(training))

Amount of training data 4482


In [73]:
print("Amount of test data",len(test))

Amount of test data 1121


In [74]:
tmod = classMeanAndStdDev(training)